In [67]:
import os
import pandas as pd
import re
import numpy as np

def parse_annotation(annot):
    list_eval_words = []
    #print ("Annot: ", str(annot[0]))
    without_spaces = str(annot[0]).replace(" ", "")
    e = re.search('/:.*/:', without_spaces)
    occurrences = without_spaces.split(";")
    #print ("Occurrences: ", occurrences)

    for occurrence in occurrences:   
        #print ("OCCURENCE DEBUG: ", occurrence) #
        #####Following are malformed strings, need to change formatting
        invalid_strings = ['炒蛋:好香+/好滑+/厚多士:勁正+','白酒蜆意粉:茨汁多左-/煙鴨胸炒飯:不過不失0/蟹肉:好多蟹肉蟹籽+/口感唔錯+','魚肉:更為嫩滑+/酒香淡淡+/朱古力:幾濃+','魚肉:嫩滑+/酒香:淡淡而發都值得加分+/朱古力:幾濃郁+/好飲+','蟹肉天使麵:好食+/火焰豬手:酥脆脆+/有質感+','叉燒包:賣相贏晒:+','蜜糖味雞蛋仔:蜜糖味偏甜搶咗蛋香-/橙朱古力雞蛋仔:唔錯+','雞柳炒飯底:好出色+/地道凍檸茶:絕配+','公仔面:夠彈+湯:鮮味+,陣陣薑味0咖央西多:新鮮+,夠熱+','中辣好辣-/米線:麻麻地-/湯底:麻麻地-/凍檸茶:太多冰','中辣好辣-/米線同湯底:麻麻地-/凍檸茶:太多冰-','濃濃湯底:正+/米線:好滑+/唔會太諗-/雲吞皮:厚咗少少-/味道都唔錯+','湯底:唔夠濃-/蝦:雪到太耐-/質地完全被破壞-/無哂應有口感-/秀珍菇:同成碗味道格格不入-', '咖啡:很濃+/洒:很香+','咖啡味:很濃+/洒味:很香+','法式焗田螺薯蓉:好食+/煙三文魚沙律:好味個汁好夾+','印度雞肉炒飯:雞肉:好好味+','韮菜:唔夠香-/韮菜粿:有少少寡-/唔夠豬油味-', '小籠包:皮薄餡靚+/素菜筍尖炒年糕:味道濃+', '鵝.蚊.件豆腐.蚊.隻蛋.共.蚊:好鹹-/豆腐:超鹹-', '饀:好足料+/餃子皮:唔會太厚+', '牛腩:好林+/好味+湯底:好飲:+河粉:好滑:+']
        #####
        if ":" in occurrence:
            if any(x in occurrence for x in invalid_strings):             
                break
            else:
                (aspect_word, eval_word_s) = occurrence.split(":")  
                #print ("Aspect_word: ", aspect_word, " eval word: ", eval_word_s)
                splitted_eval_words = process_eval_words(eval_word_s)
                list_eval_words.extend(splitted_eval_words)
        else:
            pass#print ("eval_word without aspect: ", occurrence)
            #list_eval_words.extend(occurrence)
    #print ("LIST_OF_EVAL_WORDS", list_eval_words)
        
    return (list_eval_words)


def process_eval_words(eval_word_s): # input: 十分酥嫩+/重要冇膻味+/好多肉汁+/幾好+ output: list
    # some got it wrong: 豐富+,清新+,新鮮+,濃+,甜+
    if "," in eval_word_s:
        list_of_eval_words = eval_word_s.split(",")
    else: # hoping all the rest did it correctly
        list_of_eval_words = eval_word_s.split("/")
    
    return list_of_eval_words

def compare_annotations(annot1, annot2): # now without comp-arison, just merge. TODO comparison itself!!!!
    parsed_1 = parse_annotation(annot1)
    parsed_2 = parse_annotation(annot2)
    #print ("PARSED1: ", parsed_1, "||||||| Parsed2: ", parsed_2 )
    return (parsed_1, parsed_2)
    #print ("Comparing ", without_spaces1, " and ", without_spaces2 )
    #if without_spaces1 == without_spaces2:        
    #    pass#print ("The full match!: ", without_spaces1)

def parse_frame(all_frame, columns_TODO): # After we will select not only 'taste words..'
    columns = ['taste words'] 
    words = []
    #print ("DEBUG DELETEIT, columns_TODO: ", columns_TODO)
    for group in all_frame.groupby('id'):
        #print ( "group[1].loc[:,columns].values: ", group[1].loc[:,columns].values)#.tolist() )
      
        len_group = len( group[1].loc[:,columns].values)
        list_val = group[1].loc[:,columns].values
        if len_group > 1:
            for i in range(1, len_group):
                #print ("Comparing list_val[i]= " , list_val[i], " with list_val[i-1]=", list_val[i-1])
                (annot_1, annot_2) = compare_annotations(list_val[i], list_val[i-1])
                #print ("Identical values: ", list_val[i])            
            words.extend(annot_1)
            words.extend(annot_2)
    #print ("WORDS: ", words)
    return words

if __name__ == "__main__":
    appended_annots = []
    path = 'Attachments' # The file '78065D, 71869' removed because students made quite a few malformed annotations and did not understand the task
    for filename in os.listdir(path):
        full_filename = os.path.join(path, filename)        
        data = pd.read_excel(full_filename)
        if 'id' in data.columns: #Mandarin files do not have id's        
            appended_annots.append(data)
  
    all_frame = pd.concat(appended_annots) # all annotations into one frame
    columns = ['taste words']#, 'overall words', 'environment words', 'service words', 'sanitation words']
    for column_name in columns:
        positive = set()
        negative = set()
        neutral = set()
        #taste_words = parse_frame(all_frame, column_name)
        for word in taste_words:
            if '+' in word:
                positive.add(word)
            elif '-' in word:
                negative.add(word)
            elif '0' in word:
                neutral.add(word)
    
    print ("POSITIVE: ", positive)
    print ("NEGATIVE: ", negative)
    print ("NEUTRAL: ", neutral)






POSITIVE:  {'接受到+', '滿足+', '唔會浸得太多油+', '杰度恰好+', '唔似平時食一件就覺得好溜+', '非常配合+', '仲有一陣甜味+', '唔會過多+', '必食+', '正+', '好林好味+', '但都可以+', '多汁+', '甜甜鹹鹹.加個汁就仲好食+', '新鮮+', '老少咸宜+', '正正+', '好想好想食+', '凍+', '薄+', '爽口+', '又香又滑+', '五粒星+', '值得叫呢試+', '好香松露味+', '幾生鮮+', '亦不錯+', '又真係唔錯+', '香濃+', '菜甜爽口+', '有多重口感又唔會太多+', '好清好多.都幾好味+', '肉汁豐富+', '真係唔錯+', '酸酸甜甜+', '好脆+', '唔太羶+', '值得一讚+', '不太乾身+', '汁好豐富+', '誠意之作+', '好鮮甜+', '清好多.都幾好味+', '特別+', '特別推介+', '味道濃+', '十分好+', '料都幾多+', '好味左好多+', '好爽+', '味道幾濃+', '牛奶味濃+', '有口感+', '唔錯+', '夠甜+', '好好食+', '一定要食+', '夠和味+', '冇膻味+', '夠.+', '好咬+', '幾好+', '都好有龍蝦味+', '外形特別+', '味道清清+', '有爽脆口感+', '奶味十足+', '蛋味香濃+', '都唔腥+', '好爽口+', '令人感到溫馨+', '好有口感+', '厚身之餘都好+', '都算滑+', '開胃+', '香脆+', '好過口癮+', '雞味酒味不濃+', '脆身蛋香+', '十分香口+', '又脆+', '就正正常常咁都幾香口+', '好鐘意食+', '好有蔬菜味+', '味道都唔錯+', '好清+', '甜甜地+', '係本地少數好味的+', '好濃好肥+', '係出面食唔到果隻味+', '用其他醬汁跟吃一樣好味+', '鹹甜交集又幾夾+', '味道適中+', '加埋生滾及第真係好夾+', '配脆脆食一流+', '有好味+', '+', '香香辣辣好好味+', '絕無味精+', '幾香+', '一貫水準+', '唔會太諗+', '入面有質感+', '好新鮮+', '賣相幾得意+', '火喉剛好.又新鮮.調味又好+', '落冰花+'